Trying to make the contexts match the found collocates so I can read them later for qualitative analysis.

Slightly messy and verbose + AI'd to the wazoo, but it functions

In [ ]:
import pandas as pd
import os

def process_collocates(input_csv, context_directory, output_csv=None, debug=False):
    """
    Process collocate data by merging with context files and removing duplicates.
   
    Parameters:
    -----------
    input_csv : str
        Path to the input CSV file containing collocate results
    context_directory : str
        Path to the directory containing context CSV files
    output_csv : str, optional
        Path where the output CSV should be saved. If None, no file is saved
    debug : bool, optional
        If True, print debugging information during processing
       
    Returns:
    --------
    pandas.DataFrame
        Processed DataFrame with merged context data and duplicates removed
    """
    def print_debug(message, df=None, collocate=None):
        if debug:
            print(message)
            if df is not None and collocate:
                print(f"Rows with collocate '{collocate}': {len(df[df['Collocate'] == collocate])}")
            if df is not None:
                print(f"Total rows: {len(df)}\n")
   
    # Step 1: Load the primary DataFrame and drop "Unnamed" columns
    r1 = pd.read_csv(input_csv)
    r1 = r1.loc[:, ~r1.columns.str.contains('^Unnamed')]
    print_debug("Initial data loaded:", r1, "very")
    
    # Step 2: Read all context files and combine them
    context_files = [f for f in os.listdir(context_directory) if f.endswith('.csv')]
    print_debug(f"Found {len(context_files)} context files")
   
    # Load all context files into one DataFrame
    context_data = pd.DataFrame()
    for file in context_files:
        file_path = os.path.join(context_directory, file)
        context_temp = pd.read_csv(file_path)
        context_temp = context_temp.loc[:, ~context_temp.columns.str.contains('^Unnamed')]
        context_data = pd.concat([context_data, context_temp], ignore_index=True)
    print_debug("Context data loaded:", context_data, "very")
    
    # Step 3: Merge DataFrames
    merged_df = pd.merge(r1, context_data, left_on=['Year', 'Collocate'],
                        right_on=['Year', 'Collocate'], how='left')
    print_debug("After merge, step 3:", merged_df, "very")
    
    # Step 4: Remove consecutive identical rows
    merged_df['next_year'] = merged_df['Year'].shift(-1)
    columns_to_check = [col for col in merged_df.columns if col not in ['Year']]
    mask_consecutive_identical = (
        (merged_df['next_year'] == merged_df['Year']) &
        merged_df[columns_to_check].eq(merged_df[columns_to_check].shift(-1), axis=0).all(axis=1)
    )
   
    merged_df = merged_df[~mask_consecutive_identical]
    merged_df.drop(columns=['next_year'], inplace=True)
    print_debug("After removing consecutive duplicates, step 4:", merged_df, "very")
    
    # Step 5: Remove exact duplicates
    merged_df.drop_duplicates(inplace=True)
    print_debug("After removing exact duplicates, step 5:", merged_df, "very")
    
    # Step 6: Remove duplicates keeping lowest year
    columns_to_compare = [col for col in merged_df.columns if col != 'Year']
    merged_df.sort_values(by='Year', inplace=True)
    merged_df.drop_duplicates(subset=columns_to_compare, keep='first', inplace=True)
    print_debug("After removing duplicates (keeping lowest year), step 6:", merged_df, "very")
    
    # Step 7: Remove duplicates excluding MI Score and Year #this removes some data that it shouldnt
    #columns_to_compare_mi = [col for col in merged_df.columns if col not in ['Year', 'MI Score']]
    #merged_df.sort_values(by='Year', inplace=True)
    #merged_df.drop_duplicates(subset=columns_to_compare_mi, keep='first', inplace=True)
    #print_debug("After removing duplicates (excluding MI Score):", merged_df, "very")
    
    # Step 8: Reset index
    merged_df.reset_index(drop=True, inplace=True)
    
    # Save to CSV if output path is provided
    if output_csv:
        merged_df.to_csv(output_csv, index=False)
        print_debug(f"Saved to {output_csv}")
   
    return merged_df


result_df1 = process_collocates(
    'collocate_results/dfs/fact3.csv',
    "../collocation_results/FACT/collocate_results_1665-1958_FACT_css3_w3/contexts",
    'collocate_results/contexts/fact3_contexts.csv',
    debug=True 
)

Initial data loaded:
Rows with collocate 'very': 2
Total rows: 358

Found 285 context files


In [ ]:
result_df1 = process_collocates(
    'collocate_results/dfs/fact3.csv',
    "../collocation_results/FACT/collocate_results_1665-1958_FACT_css3_w3/contexts",
    'collocate_results/contexts/fact3_contexts.csv',
    debug=True 
)

result_df2 = process_collocates(
    'collocate_results/dfs/fact5.csv',
    "../collocation_results/FACT/collocate_results_1665-1958_FACT_css3_w5/contexts",
    'collocate_results/contexts/window5/fact5_contexts.csv',
    debug=True
)

result_df3 = process_collocates(
    'collocate_results/dfs/facts3.csv',
    "../collocation_results/FACTS/collocate_results_1665-1958_FACTS_css3_w3/contexts",
    'collocate_results/contexts/facts3_contexts.csv',
    debug=True
)

result_df4 = process_collocates(
    'collocate_results/dfs/facts5.csv',
    "../collocation_results/FACTS/collocate_results_1665-1958_FACTS_css3_w5/contexts",
    'collocate_results/contexts/window5/facts5_contexts.csv',
    debug=True
)

Testing & other stuff

In [11]:
import pandas as pd
import os

# Step 1: Load the primary DataFrame and drop "Unnamed" columns (if any)
r1 = pd.read_csv('collocate_results/dfs/fact5.csv')
r1 = r1.loc[:, ~r1.columns.str.contains('^Unnamed')]  # Drop any column with 'Unnamed' in the name

# Step 2: Read all context files and combine them into a single DataFrame
context_dir =     "../collocation_results/FACT/collocate_results_1665-1958_FACT_css3_w5/contexts"
context_files = [f for f in os.listdir(context_dir) if f.endswith('.csv')]

# Load all context files into one DataFrame
context_data = pd.DataFrame()
for file in context_files:
    file_path = os.path.join(context_dir, file)
    context_temp = pd.read_csv(file_path)
    context_temp = context_temp.loc[:, ~context_temp.columns.str.contains('^Unnamed')]  # Drop "Unnamed" columns here as well
    context_data = pd.concat([context_data, context_temp], ignore_index=True)

# Step 3: Merge the primary DataFrame (r1) with the context data
merged_df = pd.merge(r1, context_data, left_on=['Year', 'Collocate'], right_on=['Year', 'Collocate'], how='left')

# Step 4: Handle NaN values
# Fill NaN values in 'Context', 'File', and 'Directory' with the previous valid value (this is later removed so a little arbitrary, but its so I can view them anyway, the differing MI Scores, if i want to)
merged_df[['Context', 'File', 'Directory']] = merged_df[['Context', 'File', 'Directory']].fillna(method='ffill')

# Step 5: Remove consecutive rows with identical values (excluding 'Year')
# Create a mask for rows where all columns except 'Year' are identical to the next row
merged_df['next_year'] = merged_df['Year'].shift(-1)
columns_to_check = [col for col in merged_df.columns if col not in ['Year']]  # Exclude 'Year'

# Compare each row with the next one, considering all columns except 'Year'
mask_consecutive_identical = (
    (merged_df['next_year'] == merged_df['Year']) &
    merged_df[columns_to_check].eq(merged_df[columns_to_check].shift(-1), axis=0).all(axis=1)
)

# Remove rows where consecutive years have identical values
merged_df = merged_df[~mask_consecutive_identical]

# Drop the temporary 'next_year' column
merged_df.drop(columns=['next_year'], inplace=True)

# Step 6: Remove exact duplicates (where all columns are identical)
merged_df.drop_duplicates(inplace=True)

# Step 7: Remove duplicates where all columns except 'Year' are the same, keeping the lowest 'Year' (same document and collocates, just slightly different years)
# First, exclude 'Year' from the columns for comparison
columns_to_compare = [col for col in merged_df.columns if col != 'Year']

# Sort the dataframe by 'Year' to ensure the lowest year comes first
merged_df.sort_values(by='Year', inplace=True)

# Drop duplicates based on all columns except 'Year', keeping the first (lowest 'Year')
merged_df.drop_duplicates(subset=columns_to_compare, keep='first', inplace=True)

# Step 8: Remove duplicates where all columns except 'MI Score' and 'Year' are the same, keeping the lowest 'Year' (same document and collocates, just slightly different years)
# Exclude 'MI Score' and 'Year' from the columns for comparison
columns_to_compare_mi = [col for col in merged_df.columns if col not in ['Year', 'MI Score']]

# Sort the dataframe by 'Year' again to ensure the lowest year comes first
merged_df.sort_values(by='Year', inplace=True)

# Drop duplicates based on all columns except 'MI Score' and 'Year', keeping the first (lowest 'Year') (same document and collocates, just slightly different years)
merged_df.drop_duplicates(subset=columns_to_compare_mi, keep='first', inplace=True)

# Step 9: Reset the index
merged_df.reset_index(drop=True, inplace=True)

print(merged_df.head())
merged_df.to_csv("test.csv", index=False)

   Year Collocate  MI Score  \
0  1670    matter      7.99   
1  1675    matter      8.09   
2  1685    matter      8.30   
3  1686    matter      8.12   
4  1694    matter      7.46   

                                             Context                File  \
0                                                NaN                 NaN   
1  irrational, if the matter of fact be true ; fo...  rstl_1675_0037.txt   
2  Philosophy , trill matter of fact and experime...  rstl_1685_0041.txt   
3  uncertainty, but the matter of FaCt be- ing mo...  rstl_1686_0017.txt   
4  now put the Matter of Fact out of Question, wh...  rstl_1694_0041.txt   

  Directory  
0       NaN  
1  txt_rstl  
2  txt_rstl  
3  txt_rstl  
4  txt_rstl  


In [7]:
def get_unique_collocates(file):
    collocates = pd.read_csv(file)['Collocate'].unique().tolist()
    print(f"\nFile: {file}")
    print(f"Collocates: {collocates}")

# Process each file
get_unique_collocates('collocate_results/contexts/fact3_contexts.csv')
get_unique_collocates('collocate_results/contexts/facts3_contexts.csv')

get_unique_collocates('collocate_results/contexts/window5/fact5_contexts.csv')
get_unique_collocates('collocate_results/contexts/window5/facts5_contexts.csv')


File: collocate_results/contexts/fact3_contexts.csv
Collocates: ['of', 'matter', 'this', 'that', 'curious', 'new', 'interesting', 'important', 'remarkable', 'in', 'they']

File: collocate_results/contexts/facts3_contexts.csv
Collocates: ['which', 'these', 'are', 'have', 'some', 'other', 'new', 'many', 'appear', 'number', 'all', 'curious', 'respecting', 'following', 'i', 'stated', 'connected', 'recorded']

File: collocate_results/contexts/window5/fact5_contexts.csv
Collocates: ['matter', 'of', 'this', 'general', 'curious', 'new', 'interesting', 'remarkable', 'important', 'that', 'due', 'attention', 'owing', 'lies', 'view', 'spite', 'do']

File: collocate_results/contexts/window5/facts5_contexts.csv
Collocates: ['these', 'have', 'some', 'prove', 'new', 'stated', 'appear', 'preceding', 'many', 'general', 'ascertained', 'chemical', 'number', 'few', 'curious', 'respecting', 'following', 'paper', 'appears', 'such', 'recorded', 'connected', 'observed', 'known', 'now', 'all']
